In [ ]:
import random
import mxnet.ndarray as nd
from mxnet import autograd as ag

In [ ]:
N = 2
M = 100000

real_w = nd.array([2, -3.4])
real_b = nd.array([4.2]).asscalar()

X = nd.random_normal(shape=(M, N))

y = nd.add(nd.dot(X, real_w), real_b)
nd.broadcast_add(y, nd.random_normal(scale=0.1, shape=y.shape), out=y)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(X[:, 1].asnumpy(), y.asnumpy())
plt.show()
plt.plot

In [ ]:
def data_grabber(X, y, batch_size = 10):
    idx = list(range(y.size))
    random.shuffle(idx)
    for _from in range(0, y.size, batch_size):
        _to = min(_from + batch_size, y.size)
        yield X[_from:_to, :], y[_from:_to]
        
        

In [ ]:
w = nd.random_normal(shape=(real_w.shape))
b = nd.zeros((1,))
params = [w, b]

for param in params:
    param.attach_grad()

In [ ]:
def net(X):
    _y = nd.add(nd.dot(X, w),
               b)
    return _y

In [ ]:
def loss_square(y_hat, y):
    _loss = nd.square(y_hat,
                     y)
    return _loss

In [ ]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

In [ ]:
def real_fn(X):
    return real_w[0] * X[:, 0] + real_w[1] * X[:, 1] + real_b
def plot(losses, X, sample_size=100):
    xs = list(range(len(losses)))
    f, (fg1, fg2) = plt.subplots(1, 2)
    fg1.set_title('Loss during training')
    fg1.plot(xs, losses, '-r')
    fg2.set_title('Estimated vs real function')
    fg2.plot(X[:sample_size, 1].asnumpy(),
             net(X[:sample_size, :]).asnumpy(), 'or', label='Estimated')
    fg2.plot(X[:sample_size, 1].asnumpy(),
             real_fn(X[:sample_size, :]).asnumpy(), '*g', label='Real')
    fg2.legend()
    plt.show()




In [ ]:

epochs = 5
learning_rate = .001
niter = 0
losses = []
moving_loss = 0
smoothing_constant = .01

# 
for e in range(epochs):
    total_loss = 0

    for data, label in data_grabber(X, y, batch_size=100):
        with ag.record():
            output = net(data)
            loss = loss_square(output, label)
        loss.backward()
        SGD(params, learning_rate)
        total_loss += nd.sum(loss).asscalar()

        niter +=1
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss

        # correct the bias from the moving averages
        est_loss = moving_loss/(1-(1-smoothing_constant)**niter)

        if (niter + 1) % 100 == 0:
            losses.append(est_loss)
            print("Epoch %s, batch %s. Moving avg of loss: %s. Average loss: %f" % (e, niter, est_loss, total_loss/M))
            plot(losses, X)